# scratch

In [2]:
import zarr
import numpy as np
from cloudvolume import CloudVolume

In [12]:
z = zarr.open("/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/sample/3-1-soma.zarr")
ng_path = "precomputed://file:///Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/sample/ng/im"

info = CloudVolume.create_new_info(
    num_channels    = 1,
    layer_type      = 'image',
    data_type       = 'uint8', # Channel images might be 'uint8'
    # raw, png, jpeg, compressed_segmentation, fpzip, kempressed, zfpc, compresso
    encoding        = 'raw', 
    resolution      = [500, 500, 300], # Voxel scaling, units are in nanometers
    voxel_offset    = [0, 0, 0], # x,y,z offset in voxels from the origin
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size      = z.shape, # units are voxels
    volume_size     = z.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(ng_path, info=info, compress = False)
vol.commit_info()
vol[:,:,:] = z[:,:,:]

In [15]:
z = zarr.open("/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/sample/3-1-soma_labels.zarr")
ng_path = "precomputed://file:///Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/sample/ng/frags"

info = CloudVolume.create_new_info(
    num_channels    = 1,
    layer_type      = 'segmentation',
    data_type       = 'uint16', # Channel images might be 'uint8'
    # raw, png, jpeg, compressed_segmentation, fpzip, kempressed, zfpc, compresso
    encoding        = 'raw', 
    resolution      = [500, 500, 300], # Voxel scaling, units are in nanometers
    voxel_offset    = [0, 0, 0], # x,y,z offset in voxels from the origin
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size      = z.shape, # units are voxels
    volume_size     = z.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(ng_path, info=info, compress = False)
vol.commit_info()
vol[:,:,:] = z[:,:,:]

Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.83it/s]
